In [1]:
from mcmc_sampler import MCMCSampler

In [2]:
sampler = MCMCSampler()

In [3]:
sampler.run_tests()

INFO:mcmc_sampler:
🧪 Running Tests on MCMC Sampler with Empirical Samples...

INFO:mcmc_sampler:🔹 Running Test: Normal Distribution Samples
INFO:mcmc_sampler:📊 Sample distribution set with 10000 points.
INFO:mcmc_sampler:   📊 Sample Distribution Set with 10000 points
INFO:mcmc_sampler:   🔧 Force Non-Negative: False
INFO:mcmc_sampler:   ✅ Completed in 1.926 sec
INFO:mcmc_sampler:   🔍 MAP Estimate: 0.059225
INFO:mcmc_sampler:   🔍 Max Value: 3.671939
INFO:mcmc_sampler:   🔍 HDI 50%: [0.000000, 0.508120]
INFO:mcmc_sampler:   🔍 HDI 95%: [0.000000, 1.952956]
INFO:mcmc_sampler:   🔍 HDI 99%: [0.000000, 2.444118]
INFO:mcmc_sampler:🔹 Running Test: Poisson Count Data
INFO:mcmc_sampler:📊 Sample distribution set with 10000 points.
INFO:mcmc_sampler:   📊 Sample Distribution Set with 10000 points
INFO:mcmc_sampler:   🔧 Force Non-Negative: True
INFO:mcmc_sampler:   ✅ Completed in 2.026 sec
INFO:mcmc_sampler:   🔍 MAP Estimate: 4.107135
INFO:mcmc_sampler:   🔍 Max Value: 13.142831
INFO:mcmc_sampler:   🔍 H